In [19]:
import os
import json

for dir in ["参数归因Cifar10", "参数归因Cifar100", "参数归因Cifar10不分label", "参数归因Cifar100不分label"]:
    run_sh = open(os.path.join(dir, "run.sh"), "w", encoding="utf-8")
    for notebook_file_name in ["main.ipynb", "minus_main.ipynb", "op_main.ipynb", "sm_main.ipynb"]:
        for thre in [0.1, 0.15, 0.2, 0.3, 0.5, 0.7]:
            py_name = notebook_file_name.split(".")[0] + "_" + str(thre) + ".py"
            run_sh.write(f"python {py_name}" + "\n")
            py_path = os.path.join(dir, py_name)
            notebook_file = open(os.path.join(
                dir, notebook_file_name), "r", encoding="utf-8")
            notebook_file = json.load(notebook_file)
            with open(py_path, "w", encoding="utf-8") as f:
                for idx, cell in enumerate(notebook_file["cells"]):
                    f.write("\n")
                    f.write(r"# %%")
                    f.write("\n")
                    for elem in cell["source"]:
                        if "temp / all_num" in elem:
                            elem = 'log_file.write("保留率: " + str(temp / all_num) + "\\n")\n'
                        elif 'print("原始准确率"' in elem:
                            elem = '    log_file.write("原始准确率: " + str((preds.argmax(-1) == labels).mean()) + "\\n")\n'
                        elif 'print("现在准确率"' in elem:
                            elem = '    log_file.write("现在准确率: " + str((preds.argmax(-1) == labels).mean()) + "\\n")\n'
                        elif 'print("对比实验准确率"' in elem:
                            elem = '    log_file.write("对比实验准确率: " + str((preds.argmax(-1) == labels).mean()) + "\\n")\n'
                        elif "pkl.dump(all_totals" in elem:
                            elem = elem.split('.')
                            elem = ".".join(elem[:-1]) + f"_{thre}" + "." + elem[-1]
                            # pkl.dump(all_totals, open("weights/minus_totals.pkl", "wb"))
                        elif elem.startswith("thre = "):
                            elem = f"thre = {thre}\n"
                        f.write(elem)
                    if idx == 0:
                        f.write("\nimport os\n")
                        f.write("file_name = os.path.basename(__file__)\n")
                        f.write(
                            'log_name = file_name.replace("py","log")\n')
                        f.write('log_file = open(log_name, "w")\n')
                f.write("\nlog_file.close()\n")
            f.close()
    run_sh.close()
            # os.system(f"autopep8 {py_path} --in-place")

#     with open("test.py","w",encoding="utf-8") as f:
#         for idx,cell in enumerate(notebook_file["cells"]):
#             f.write(r"# %%")
#             f.write("\n")
#             for elem in cell["source"]:
#                 if elem == "temp / all_num\n":
#                     elem = 'log_file.write("保留率: " + str(temp / all_num) + "\\n")\n'
#                 elif elem == '    print("原始准确率", (preds.argmax(-1) == labels).mean())\n':
#                     elem = '    log_file.write("原始准确率: " + str((preds.argmax(-1) == labels).mean()) + "\\n")\n'
#                 elif elem == '    print("现在准确率", (preds.argmax(-1) == labels).mean())\n':
#                     elem = '    log_file.write("现在准确率: " + str((preds.argmax(-1) == labels).mean()) + "\\n")\n'
#                 elif elem == '    print("对比实验准确率", (preds.argmax(-1) == labels).mean())\n':
#                     elem = '    log_file.write("对比实验准确率: " + str((preds.argmax(-1) == labels).mean()) + "\\n")\n'
#                 f.write(elem)
#             if idx == 0:
#                 f.write("\nimport os\n")
#                 f.write("file_name = os.path.basename(__file__)\n")
#                 f.write('log_name = file_name.split(".")[0] + ".log"\n')
#                 f.write('log_file = open(log_name, "w")\n')
#         f.write("\nlog_file.close()\n")
#     f.close()
# # os.system(f"autopep8 test.py --in-place")
